In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import os
import random
from PyPDF2 import PdfFileReader

In [13]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
browser = webdriver.Chrome('/Users/senlinlidewo/Library/Application Support/Google/Chrome/chromedriver',options=chrome_options)
#os.makedirs('D:\\公司年报')  #在D盘创建文件夹，若重复运行，注释此行

#解析网址
def get_html_content(url):
    header = {
        "User-Agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Mobile Safari/537.36"
        }
    r = requests.get(url,headers = header)
    if r.status_code == 200:
        r.encoding = 'utf-8'
        #print(r.content)
        return r.content
    else:
        return None
  
#保存pdf   
def report_save(url,pdf_name):
    report = get_html_content(url)
    path = ".\" + pdf_name + ".pdf"
    with open(path,'wb') as f:
        f.write(report)
        f.close
        
#获取年报页数
def get_num_pages(pdf_name):
    path = ".\" + pdf_name + ".pdf"
    reader = PdfFileReader(path)
    if reader.isEncrypted:
        reader.decrypt('')
    page_num = reader.getNumPages()
    return page_num

url = 'http://www.cninfo.com.cn/new/commonUrl/pageOfSearch?url=disclosure/list/search&checkedCategory=category_ndbg_szsh#sse'
browser.get(url)

#修改财报公布日期期间
browser.find_element_by_xpath('//*[@id="main"]/div[2]/div[1]/div[2]/div[1]/div[2]/form/div[1]/div/div/i[1]').click() 
browser.find_element_by_xpath('/html/body/div[5]/div[1]/div[1]/button[7]').send_keys(Keys.ENTER) 

items = []
count = 1
while count<=1:
    count += 1
    time.sleep(2)
    all_tr = browser.find_element_by_xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[2]/div/div[3]/table/tbody').find_elements_by_xpath('.//tr')
    for tr in all_tr:
        item = {}
        a=random.random()*3
        time.sleep(a)   #设置随机休息
        number = tr.find_elements_by_xpath('./td')[0].find_element_by_xpath('.//span').text
        item['公司代码'] = number
        name_pre = tr.find_elements_by_xpath('./td')[1].find_element_by_xpath('.//span').text
        name = name_pre.replace('*','')  #windows命名不能包含*，去除*
        item['公司名称'] = name
        title = tr.find_elements_by_xpath('./td')[2].find_element_by_xpath('.//a').text
        item['年报标题'] = title
        date = tr.find_elements_by_xpath('./td')[3].find_element_by_xpath('.//span').text
        item['年报发布日期'] = date
        year = str(int(date[0:4])-1)
        item['年报对应年份'] = year
        pdf_name = number + "-" + name + "-" + year   #设置pdf命名
        print(pdf_name)
        handle_main = browser.current_window_handle   #句柄
        tr.find_elements_by_xpath('./td')[2].find_element_by_xpath('.//a').send_keys(Keys.ENTER)  #点击进入网页
        if len(browser.window_handles)>1:
            for handle in browser.window_handles:
                if handle != handle_main:
                    browser.switch_to.window(handle)   #切换到新句柄
                    pdf_href = browser.find_element_by_xpath('//*[@id="noticeDetail"]/div/div[2]/div[1]/a').get_attribute('href')   #获取pdf网址
                    print(pdf_href)
                    report_save(pdf_href,pdf_name)
                    browser.close()
        browser.switch_to.window(handle_main)  #切换回主句柄
        page_num = get_num_pages(pdf_name)
        print(page_num)
        item['年报页数'] = page_num
        items.append(item)
        
    if browser.find_element_by_xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[3]/div/button[2]').get_attribute('disabled') ==  'disabled':
        break
    browser.find_element_by_xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[3]/div/button[2]/i').click()

df = pd.DataFrame(items)
df.to_excel(r'./上市公司年报.xlsx')


SyntaxError: EOL while scanning string literal (<ipython-input-13-a6f798ea355f>, line 22)

In [22]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from lxml import etree
import csv
import os
from time import sleep
from random import randint

In [42]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
browser = webdriver.Chrome('/Users/senlinlidewo/Library/Application Support/Google/Chrome/chromedriver',options = chrome_options)
#url = 'http://paiming.funds.hexun.com/pm/'
url = 'http://webapi.cninfo.com.cn/#/aiInfos?sname=000001'
browser.get(url)
data = browser.page_source

In [46]:
table = pd.read_html(data)[0]
table

,发布时间,证券代码,资讯标题,资讯类别
0,2021-02-24 21:28:27,1,华夏成长(000001)：基金经理变动，新任基金经理：阳琨。,基金经理变动
1,2021-02-01 18:57:10,1,平安银行(000001)最新公布2020年年报。,年报
2,2020-10-27 12:36:38,1,华夏成长：2020年第三季度报告，期内净值增长率为6.53%,季报
3,2020-10-26 16:58:07,1,平安银行：2020年第三季度报告正文,三季报
4,2020-08-27 17:47:31,1,平安银行：2020年半年度报告,半年报
5,2020-05-21 17:26:04,1,平安银行：2019年年度权益分派实施，每10股派发现金红利2.18元（含税）,分红
6,2020-04-23 08:46:50,1,华夏成长：2020年第一季度报告，期内净值增长率为-1.63%,季报
7,2020-04-14 20:32:31,1,平安银行：选举产生职工监事,高管变更
8,2020-04-09 13:56:46,1,华夏成长：2019年年度报告，期内基金份额净值增长率为25.97%,年报
9,2020-02-13 19:00:04,1,平安银行最新公布2019年年报,年报


In [18]:
url = 'http://webapi.cninfo.com.cn/#/aiInfos?sname=000001'
headers = {'user-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}